In [3]:
import pandas as pd
import numpy as np
#import seaborn as sns

In [17]:
# importing R dependencies
import rpy2
import readline
import rpy2.robjects as robjects

In [18]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [19]:
robjects.r('pi')

array([ 3.14159265])

In [22]:
%R require(ggplot2)

/home/mattreyes/anaconda3/lib/python3.5/site-packages/rpy2/rinterface/__init__.py:185: RRuntimeWarning: Loading required package: ggplot2

  warnings.warn(x, RRuntimeWarning)


array([0], dtype=int32)

In [26]:
robjects.r.array([1])

R object with classes: ('array',) mapped to:
<Array - Python:0x7fa50d6e4dc8 / R:0x30a3b38>
[IntVector]

Algorithm

In [3]:
class Particle:
    def __init__(self,position,velocity):
        self.position = position
        self.velocity = velocity
        self.pbest = position
        self.current_fitness = calculate_fitness(self.position)
        self.best_fitness = self.current_fitness
    def calculate_fitness(self,position):
        #self.current_fitness = gaussian formula
        if self.current_fitness > self.best_fitness:
            self.pbest = self.position
            self.best_fitness = self.current_fitness
    def update_position(self,position,pbest,gbest):
        c1 = 2
        c2 = 2
        r1 = np.random.rand(1)
        r2 = np.random.rand(1)
        self.velocity = self.velocity + 2*r1*(pbest - position) + 2*r2*(gbest - position)
        self.position = self.position + self.velocity
        
# call update before calculation

In [ ]:
# Algorithm

N = 1000
iterations = 50000

#for i in range(iterations):
    
    
# initialize partciels (list of classes)


    # random posn (based on mean)
    # random velocity
    
    
    
# globally, take the max pbest



In [70]:
v = np.array([1,2])
r = np.random.rand(1)
print(v)
print(r)
v * r


[1 2]
[ 0.5903245]


array([ 0.5903245,  1.180649 ])